**CSE 143 Assignment 3**

# Setup

Sourced from the starter code, we will import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed, as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 33.1 MB/s eta 0:00:00


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


# Sentiment Analysis: Classification with RNNs

In [ ]:
tf.random.set_seed(42)

You can load the IMDB dataset easily:

In [ ]:
(X_train, y_test), (X_valid, y_test) = keras.datasets.imdb.load_data()

17464789/17464789 [==============================] - 2s 0us/step


In [ ]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [ ]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

1641221/1641221 [==============================] - 1s 1us/step


'<sos> this film was just brilliant casting location scenery story'

In [ ]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDDV2HJ/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDDV2HJ/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDDV2HJ/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
datasets.keys()

dict_keys([Split('train'), Split('test'), Split('unsupervised')])

In [ ]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [ ]:
train_size, test_size

(25000, 25000)

In [ ]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



In [ ]:
# Truncates the input strings to their first 300 characters.
# Replaces any HTML line break tags (<br> or <br />) with a space character.
# Replaces any character that is not a letter or an apostrophe with a space character.
# Splits the strings into a ragged tensor of substrings.
# Converts the ragged tensor to a dense tensor, padding any shorter sequences with the default value "<pad>".
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [ ]:
preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'having', b'j

In [ ]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [ ]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [ ]:
len(vocabulary)

53893

In [ ]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [ ]:
# Embed the input text as a sequence of vectors?
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaaantastic".split():
    print(word_to_id.get(word) or vocab_size)

22
12
11
10000


In [ ]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [ ]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]])>

In [ ]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].repeat().batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [ ]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

tf.Tensor(
[[  22   11   28 ...    0    0    0]
 [   6   21   70 ...    0    0    0]
 [4099 6881    1 ...    0    0    0]
 ...
 [  22   12  118 ...  331 1047    0]
 [1757 4101  451 ...    0    0    0]
 [3365 4392    6 ...    0    0    0]], shape=(32, 60), dtype=int64)
tf.Tensor([0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0], shape=(32,), dtype=int64)


In [ ]:
# num of layers for the RNN
# embed_size = 128
# model = keras.models.Sequential([
#     keras.layers.Dense(1, activation="sigmoid")
# ])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# history = model.fit(train_set, steps_per_epoch=train_size // 32, epochs=20)

from keras.layers import Embedding, Dense, Dropout, Input
from keras.models import Model

model = keras.models.Sequential([
    keras.layers.Embedding(train_size, 16, input_shape=[None]),
    keras.layers.SimpleRNN(32, dropout=0.5),
    #keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="sigmoid")
])

# input_layer = Input(shape=(300))
# x = Embedding(train_size, 16)(input_layer)
# x = Dropout(0.5)(x)
# x = Dense(1, activation='tanh')(x)
# new_model = Model(input_layer, x)

loss = 'binary_crossentropy'

opt = 'adam'

metrics = 'accuracy'

model.compile(loss=loss, optimizer=opt, metrics=metrics)

model.summary()


batchsize = 32
epochs = 20

# Fit model
history = model.fit(train_set, epochs=epochs,steps_per_epoch = train_size//32)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 16)          400000    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                1568      
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 401,601
Trainable params: 401,601
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
781/781 [==============================] - 85s 105ms/step - loss: 0.6949 - accuracy: 0.5077
Epoch 2/20
781/781 [==============================] - 66s 85ms/step - loss: 0.6936 - accuracy: 0.5104
Epoch 3/20
781/781 [==============================] - 67s 85ms/step - loss: 0.6865 - accuracy: 0.5450
Epoch

In [ ]:
# Model: "sequential_3"
# _________________________________________________________________
#  Layer (type)                Output Shape              Param #
# =================================================================
#  embedding_4 (Embedding)     (None, None, 16)          400000

#  simple_rnn_1 (SimpleRNN)    (None, 32)                1568

#  dense_4 (Dense)             (None, 1)                 33

# =================================================================
# Total params: 401,601
# Trainable params: 401,601
# Non-trainable params: 0
# _________________________________________________________________
# Epoch 1/20
# 781/781 [==============================] - 85s 105ms/step - loss: 0.6949 - accuracy: 0.5077
# Epoch 2/20
# 781/781 [==============================] - 66s 85ms/step - loss: 0.6936 - accuracy: 0.5104
# Epoch 3/20
# 781/781 [==============================] - 67s 85ms/step - loss: 0.6865 - accuracy: 0.5450
# Epoch 4/20
# 781/781 [==============================] - 66s 84ms/step - loss: 0.5980 - accuracy: 0.6886
# Epoch 5/20
# 781/781 [==============================] - 65s 83ms/step - loss: 0.4754 - accuracy: 0.7840
# Epoch 6/20
# 781/781 [==============================] - 66s 85ms/step - loss: 0.4100 - accuracy: 0.8249
# Epoch 7/20
# 781/781 [==============================] - 66s 84ms/step - loss: 0.3675 - accuracy: 0.8473
# Epoch 8/20
# 781/781 [==============================] - 66s 84ms/step - loss: 0.3413 - accuracy: 0.8591
# Epoch 9/20
# 781/781 [==============================] - 64s 82ms/step - loss: 0.3326 - accuracy: 0.8665
# Epoch 10/20
# 781/781 [==============================] - 66s 84ms/step - loss: 0.3166 - accuracy: 0.8714
# Epoch 11/20
# 781/781 [==============================] - 67s 85ms/step - loss: 0.2913 - accuracy: 0.8846
# Epoch 12/20
# 781/781 [==============================] - 65s 83ms/step - loss: 0.2888 - accuracy: 0.8840
# Epoch 13/20
# 781/781 [==============================] - 65s 84ms/step - loss: 0.2772 - accuracy: 0.8920
# Epoch 14/20
# 781/781 [==============================] - 64s 81ms/step - loss: 0.2598 - accuracy: 0.9001
# Epoch 15/20
# 781/781 [==============================] - 67s 86ms/step - loss: 0.2532 - accuracy: 0.9017
# Epoch 16/20
# 781/781 [==============================] - 68s 87ms/step - loss: 0.2485 - accuracy: 0.9045
# Epoch 17/20
# 781/781 [==============================] - 67s 86ms/step - loss: 0.2415 - accuracy: 0.9060
# Epoch 18/20
# 781/781 [==============================] - 70s 89ms/step - loss: 0.2360 - accuracy: 0.9099
# Epoch 19/20
# 781/781 [==============================] - 67s 86ms/step - loss: 0.2290 - accuracy: 0.9133
# Epoch 20/20
# 764/781 [============================>.] - ETA: 1s - loss: 0.2244 - accuracy: 0.9142

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.layers import Embedding, Dense, Dropout, Input
from keras.models import Model
model = keras.models.Sequential([
    keras.layers.Embedding(train_size, 16, input_shape=[None]),
    keras.layers.LSTM(32, dropout=0.5),
    #keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="sigmoid")
])

# input_layer = Input(shape=(300))
# x = Embedding(train_size, 16)(input_layer)
# x = Dropout(0.5)(x)
# x = Dense(1, activation='tanh')(x)
# new_model = Model(input_layer, x)

loss = 'binary_crossentropy'

opt = 'adam'

metrics = 'accuracy'

model.compile(loss=loss, optimizer=opt, metrics=metrics)

model.summary()


batchsize = 32
epochs = 20

# Fit model
history = model.fit(train_set, epochs=epochs,steps_per_epoch = train_size//32)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          400000    
                                                                 
 lstm (LSTM)                 (None, 32)                6272      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 406,305
Trainable params: 406,305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
781/781 [==============================] - 40s 42ms/step - loss: 0.5623 - accuracy: 0.6977
Epoch 2/20
781/781 [==============================] - 8s 11ms/step - loss: 0.4113 - accuracy: 0.8177
Epoch 3/20
781/781 [==============================] - 7s 9ms/step - loss: 0.3525 - accuracy: 0.8515
Epoch 4/20


In [ ]:
#1.2: Model: "sequential"
# _________________________________________________________________
#  Layer (type)                Output Shape              Param #
# =================================================================
#  embedding (Embedding)       (None, None, 16)          400000

#  lstm (LSTM)                 (None, 32)                6272

#  dense (Dense)               (None, 1)                 33

# =================================================================
# Total params: 406,305
# Trainable params: 406,305
# Non-trainable params: 0
# _________________________________________________________________
# Epoch 1/20
# 781/781 [==============================] - 40s 42ms/step - loss: 0.5623 - accuracy: 0.6977
# Epoch 2/20
# 781/781 [==============================] - 8s 11ms/step - loss: 0.4113 - accuracy: 0.8177
# Epoch 3/20
# 781/781 [==============================] - 7s 9ms/step - loss: 0.3525 - accuracy: 0.8515
# Epoch 4/20
# 781/781 [==============================] - 7s 9ms/step - loss: 0.3305 - accuracy: 0.8651
# Epoch 5/20
# 781/781 [==============================] - 6s 7ms/step - loss: 0.3033 - accuracy: 0.8768
# Epoch 6/20
# 781/781 [==============================] - 7s 8ms/step - loss: 0.2874 - accuracy: 0.8836
# Epoch 7/20
# 781/781 [==============================] - 6s 7ms/step - loss: 0.2671 - accuracy: 0.8933
# Epoch 8/20
# 781/781 [==============================] - 7s 8ms/step - loss: 0.2557 - accuracy: 0.8978
# Epoch 9/20
# 781/781 [==============================] - 6s 8ms/step - loss: 0.2446 - accuracy: 0.9027
# Epoch 10/20
# 781/781 [==============================] - 7s 9ms/step - loss: 0.2329 - accuracy: 0.9087
# Epoch 11/20
# 781/781 [==============================] - 6s 7ms/step - loss: 0.2274 - accuracy: 0.9113
# Epoch 12/20
# 781/781 [==============================] - 7s 9ms/step - loss: 0.2169 - accuracy: 0.9152
# Epoch 13/20
# 781/781 [==============================] - 6s 7ms/step - loss: 0.2107 - accuracy: 0.9194
# Epoch 14/20
# 781/781 [==============================] - 7s 9ms/step - loss: 0.2017 - accuracy: 0.9222
# Epoch 15/20
# 781/781 [==============================] - 6s 7ms/step - loss: 0.1950 - accuracy: 0.9254
# Epoch 16/20
# 781/781 [==============================] - 7s 8ms/step - loss: 0.1898 - accuracy: 0.9279
# Epoch 17/20
# 781/781 [==============================] - 6s 7ms/step - loss: 0.1858 - accuracy: 0.9278
# Epoch 18/20
# 781/781 [==============================] - 7s 8ms/step - loss: 0.1768 - accuracy: 0.9321
# Epoch 19/20
# 781/781 [==============================] - 6s 7ms/step - loss: 0.1706 - accuracy: 0.9352
# Epoch 20/20
# 781/781 [==============================] - 7s 8ms/step - loss: 0.1690 - accuracy: 0.9365

# Viterbi Algorithm

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import os
import sys
print(os.listdir('/content/gdrive/My Drive/CSE-143-A3'))
sys.path.append('/content/gdrive/My Drive/CSE-143-A3')

Mounted at /content/gdrive/
['ner.dev', 'ner.test', 'model.simple', 'conlleval.py', 'ner.train', '__pycache__', 'ner.dev.out']


In [ ]:
%cd '/content/gdrive/My Drive/CSE-143-A3'

/content/gdrive/My Drive/CSE-143-A3


In [ ]:
import random
from conlleval import evaluate as conllevaluate
import numpy as np
directory = '/content/gdrive/My Drive/CSE-143-A3'

def decode(input_length, tagset, score):
    """
    Compute the highest scoring sequence according to the scoring function.
    :param input_length: int. number of tokens in the input including <START> and <STOP>
    :param tagset: Array of strings, which are the possible tags.  Does not have <START>, <STOP>
    :param score: function from current_tag (string), previous_tag (string), i (int) to the score.  i=0 points to
        <START> and i=1 points to the first token. i=input_length-1 points to <STOP>
    :return: Array strings of length input_length, which is the highest scoring tag sequence including <START> and <STOP>
    """
    # Look at the function compute_score for an example of how the tag sequence should be scored
    v = [] # viterbi values
    b = [] # backpointers for viterbi values
    y = [] # best path
    # populating the matrices and path with empty values
    for x in range(input_length):
        v.append([0]*len(tagset))
        b.append([0]*len(tagset))
        y += [""]
    # Filling in initial values for first viterbi iteration with start score
    for k in range(len(tagset)):
        v[1][k] = score(tagset[k],"<START>",1)
    for m in range(2,input_length):
        for k in range(len(tagset)):
            greatest = None
            index = None
            # find greatest value and index of said value for each combination of current iteration
            for l in range(len(tagset)):
                value = (score(tagset[k],tagset[l],m)) + v[m-1][l]
                if greatest is None or value > greatest:
                    greatest = value
                    index = l
            # insert the values into the viterbi values and indexes into backpointers
            v[m][k] = greatest
            b[m][k] = index
    greatest = None
    index = None
    for k in range(len(tagset)):
        # find greatest values for the last iteration aka the stop token
        value = score("<STOP>", tagset[k], input_length - 1) + v[input_length - 1][k]
        if greatest is None or value > greatest:
            greatest = value
            index = k

    # set the start token to the first value
    y[0] = "<START>"
    # populate path with indexes of found in backtags for that iteration, working backwards
    for m in range(input_length-1,0,-1):
        y[m] = tagset[index]
        index = b[m][index]
    #set the end token to the last value
    print(y)
    y[-1] = "<STOP>"

    return y
    #raise NotImplementedError("Complete Viterbi Decoding Here!")

def compute_score(tag_seq, input_length, score):
    """
    Computes the total score of a tag sequence
    :param tag_seq: Array of String of length input_length. The tag sequence including <START> and <STOP>
    :param input_length: Int. input length including the padding <START> and <STOP>
    :param score: function from current_tag (string), previous_tag (string), i (int) to the score.  i=0 points to
        <START> and i=1 points to the first token. i=input_length-1 points to <STOP>
    :return:
    """
    total_score = 0
    for i in range(1, input_length):
        total_score += score(tag_seq[i], tag_seq[i - 1], i)
    return total_score


def compute_features(tag_seq, input_length, features):
    """
    Compute f(xi, yi)
    :param tag_seq: [tags] already padded with <START> and <STOP>
    :param input_length: input length including the padding <START> and <STOP>
    :param features: func from token index to FeatureVector
    :return:
    """
    feats = FeatureVector({})
    for i in range(1, input_length):
        feats.times_plus_equal(1, features.compute_features(tag_seq[i], tag_seq[i - 1], i))
    return feats


def sgd(training_size, epochs, gradient, parameters, training_observer):
    """
    Stochastic gradient descent
    :param training_size: int. Number of examples in the training set
    :param epochs: int. Number of epochs to run SGD for
    :param gradient: func from index (int) in range(training_size) to a FeatureVector of the gradient
    :param parameters: FeatureVector.  Initial parameters.  Should be updated while training
    :param training_observer: func that takes epoch and parameters.  You can call this function at the end of each
           epoch to evaluate on a dev set and write out the model parameters for early stopping.
    :return: final parameters
    """
    # Look at the FeatureVector object.  You'll want to use the function times_plus_equal to update the
    # parameters.
    # To implement early stopping you can call the function training_observer at the end of each epoch.
    i = 0

    while i < epochs:
        print('i='+str(i))
        data_indices = [ i for i in range(training_size) ]
        random.shuffle(data_indices)
        counter = 0
        for t in data_indices:
            if counter % 1000 == 0:
                print('Item '+str(counter))
            parameters.times_plus_equal(-1, gradient(t))
            counter += 1
        i += 1
        training_observer(i, parameters)
    return


def train(data, feature_names, tagset, epochs):
    """
    Trains the model on the data and returns the parameters
    :param data: Array of dictionaries representing the data.  One dictionary for each data point (as created by the
        make_data_point function).
    :param feature_names: Array of Strings.  The list of feature names.
    :param tagset: Array of Strings.  The list of tags.
    :param epochs: Int. The number of epochs to train
    :return: FeatureVector. The learned parameters.
    """
    parameters = FeatureVector({})

    def perceptron_gradient(i):
        """
        Computes the gradient of the Perceptron loss for example i
        :param i: Int
        :return: FeatureVector
        """
        inputs = data[i]
        input_len = len(inputs['tokens'])
        gold_labels = inputs['gold_tags']
        features = Features(inputs, feature_names)

        def score(cur_tag, pre_tag, i):
            return parameters.dot_product(features.compute_features(cur_tag, pre_tag, i))

        tags = decode(input_len, tagset, score)
        fvector = compute_features(tags, input_len, features)           # Add the predicted features
        #print('Input:', inputs)        # helpful for debugging
        #print("Predicted Feature Vector:", fvector.fdict)
        #print("Predicted Score:", parameters.dot_product(fvector))
        fvector.times_plus_equal(-1, compute_features(gold_labels, input_len, features))    # Subtract the features for the gold labels
        #print("Gold Labels Feature Vector: ", compute_features(gold_labels, input_len, features).fdict)
        #print("Gold Labels Score:", parameters.dot_product(compute_features(gold_labels, input_len, features)))
        return fvector

    def training_observer(epoch, parameters):
        """
        Evaluates the parameters on the development data, and writes out the parameters to a 'model.iter'+epoch and
        the predictions to 'ner.dev.out'+epoch.
        :param epoch: int.  The epoch
        :param parameters: Feature Vector.  The current parameters
        :return: Double. F1 on the development data
        """
        dev_data = read_data('ner.dev')
        (_, _, f1) = evaluate(dev_data, parameters, feature_names, tagset)
        write_predictions('ner.dev.out'+str(epoch), dev_data, parameters, feature_names, tagset)
        parameters.write_to_file('model.iter'+str(epoch))
        return f1

    return sgd(len(data), epochs, perceptron_gradient, parameters, training_observer)


def predict(inputs, input_len, parameters, feature_names, tagset):
    """

    :param inputs:
    :param input_len:
    :param parameters:
    :param feature_names:
    :param tagset:
    :return:
    """
    features = Features(inputs, feature_names)

    def score(cur_tag, pre_tag, i):
        return parameters.dot_product(features.compute_features(cur_tag, pre_tag, i))

    return decode(input_len, tagset, score)


def make_data_point(sent):
    """
        Creates a dictionary from String to an Array of Strings representing the data.  The dictionary items are:
        dic['tokens'] = Tokens padded with <START> and <STOP>
        dic['pos'] = POS tags padded with <START> and <STOP>
        dic['NP_chunk'] = Tags indicating noun phrase chunks, padded with <START> and <STOP>
        dic['gold_tags'] = The gold tags padded with <START> and <STOP>
    :param sent: String.  The input CoNLL format string
    :return: Dict from String to Array of Strings.
    """
    dic = {}
    sent = [s.strip().split() for s in sent]
    dic['tokens'] = ['<START>'] + [s[0] for s in sent] + ['<STOP>']
    dic['pos'] = ['<START>'] + [s[1] for s in sent] + ['<STOP>']
    dic['NP_chunk'] = ['<START>'] + [s[2] for s in sent] + ['<STOP>']
    dic['gold_tags'] = ['<START>'] + [s[3] for s in sent] + ['<STOP>']
    return dic

def read_data(filename):
    """
    Reads the CoNLL 2003 data into an array of dictionaries (a dictionary for each data point).
    :param filename: String
    :return: Array of dictionaries.  Each dictionary has the format returned by the make_data_point function.
    """
    data = []
    with open(filename, 'r') as f:
        sent = []
        for line in f.readlines():
            if line.strip():
                sent.append(line)
            else:
                data.append(make_data_point(sent))
                sent = []
        data.append(make_data_point(sent))

    return data

def write_predictions(out_filename, all_inputs, parameters, feature_names, tagset):
    """
    Writes the predictions on all_inputs to out_filename, in CoNLL 2003 evaluation format.
    Each line is token, pos, NP_chuck_tag, gold_tag, predicted_tag (separated by spaces)
    Sentences are separated by a newline
    The file can be evaluated using the command: python conlleval.py < out_file
    :param out_filename: filename of the output
    :param all_inputs:
    :param parameters:
    :param feature_names:
    :param tagset:
    :return:
    """
    with open(out_filename, 'w', encoding='utf-8') as f:
        for inputs in all_inputs:
            input_len = len(inputs['tokens'])
            tag_seq = predict(inputs, input_len, parameters, feature_names, tagset)
            for i, tag in enumerate(tag_seq[1:-1]):  # deletes <START> and <STOP>
                f.write(' '.join([inputs['tokens'][i+1], inputs['pos'][i+1], inputs['NP_chunk'][i+1], inputs['gold_tags'][i+1], tag])+'\n') # i + 1 because of <START>
            f.write('\n')

def evaluate(data, parameters, feature_names, tagset):
    """
    Evaluates precision, recall, and F1 of the tagger compared to the gold standard in the data
    :param data: Array of dictionaries representing the data.  One dictionary for each data point (as created by the
        make_data_point function)
    :param parameters: FeatureVector.  The model parameters
    :param feature_names: Array of Strings.  The list of features.
    :param tagset: Array of Strings.  The list of tags.
    :return: Tuple of (prec, rec, f1)
    """
    all_gold_tags = [ ]
    all_predicted_tags = [ ]
    for inputs in data:
        all_gold_tags.extend(inputs['gold_tags'][1:-1])  # deletes <START> and <STOP>
        input_len = len(inputs['tokens'])
        all_predicted_tags.extend(predict(inputs, input_len, parameters, feature_names, tagset)[1:-1]) # deletes <START> and <STOP>
    return conllevaluate(all_gold_tags, all_predicted_tags)


def test_decoder():
    # See

    tagset = ['NN', 'VB']     # make up our own tagset

    def score_wrap(cur_tag, pre_tag, i):
        retval = score(cur_tag, pre_tag, i)
        print('Score('+cur_tag+','+pre_tag+','+str(i)+') returning '+str(retval))
        return retval

    def score(cur_tag, pre_tag, i):
        if i == 0:
            print("ERROR: Don't call score for i = 0 (that points to <START>, with nothing before it)")
        if i == 1:
            if pre_tag != '<START>':
                print("ERROR: Previous tag should be <START> for i = 1. Previous tag = "+pre_tag)
            if cur_tag == 'NN':
                return 6
            if cur_tag == 'VB':
                return 4
        if i == 2:
            if cur_tag == 'NN' and pre_tag == 'NN':
                return 4
            if cur_tag == 'NN' and pre_tag == 'VB':
                return 9
            if cur_tag == 'VB' and pre_tag == 'NN':
                return 5
            if cur_tag == 'VB' and pre_tag == 'VB':
                return 0
        if i == 3:
            if cur_tag != '<STOP>':
                print('ERROR: Current tag at i = 3 should be <STOP>. Current tag = '+cur_tag)
            if pre_tag == 'NN':
                return 1
            if pre_tag == 'VB':
                return 1

    predicted_tag_seq = decode(4, tagset, score_wrap)
    print('Predicted tag sequence should be = <START> VB NN <STOP>')
    print('Predicted tag sequence = '+' '.join(predicted_tag_seq))
    print("Score of ['<START>','VB','NN','<STOP>'] = "+str(compute_score(['<START>','VB','NN','<STOP>'], 4, score)))
    print('Max score should be = 14')
    print('Max score = '+str(compute_score(predicted_tag_seq, 4, score)))


def main_predict(data_filename, model_filename):
    """
    Main function to make predictions.
    Loads the model file and runs the NER tagger on the data, writing the output in CoNLL 2003 evaluation format to data_filename.out
    :param data_filename: String
    :param model_filename: String
    :return: None
    """
    data = read_data(data_filename)
    parameters = FeatureVector({})
    parameters.read_from_file(model_filename)

    tagset = ['B-PER', 'B-LOC', 'B-ORG', 'B-MISC', 'I-PER', 'I-LOC', 'I-ORG', 'I-MISC', 'O']
    feature_names = ['tag', 'prev_tag', 'current_word']

    write_predictions(data_filename+'.out', data, parameters, feature_names, tagset)
    evaluate(data, parameters, feature_names, tagset)

    return


def main_train():
    """
    Main function to train the model
    :return: None
    """
    print('Reading training data')
    train_data = read_data('ner.train')
    #train_data = read_data('ner.train')[1:1] # if you want to train on just one example

    tagset = ['B-PER', 'B-LOC', 'B-ORG', 'B-MISC', 'I-PER', 'I-LOC', 'I-ORG', 'I-MISC', 'O']
    feature_names = ['tag', 'prev_tag', 'current_word']

    print('Training...')
    parameters = train(train_data, feature_names, tagset, epochs=10)
    print('Training done')
    dev_data = read_data('ner.dev')
    evaluate(dev_data, parameters, feature_names, tagset)
    test_data = read_data('ner.test')
    evaluate(test_data, parameters, feature_names, tagset)
    parameters.write_to_file('model')

    return


class Features(object):
    def __init__(self, inputs, feature_names):
        """
        Creates a Features object
        :param inputs: Dictionary from String to an Array of Strings.
            Created in the make_data_point function.
            inputs['tokens'] = Tokens padded with <START> and <STOP>
            inputs['pos'] = POS tags padded with <START> and <STOP>
            inputs['NP_chunk'] = Tags indicating noun phrase chunks, padded with <START> and <STOP>
            inputs['gold_tags'] = DON'T USE! The gold tags padded with <START> and <STOP>
        :param feature_names: Array of Strings.  The list of features to compute.
        """
        self.feature_names = feature_names
        self.inputs = inputs

    def compute_features(self, cur_tag, pre_tag, i):
        """
        Computes the local features for the current tag, the previous tag, and position i
        :param cur_tag: String.  The current tag.
        :param pre_tag: String.  The previous tag.
        :param i: Int. The position
        :return: FeatureVector
        """
        feats = FeatureVector({})
        if 'tag' in self.feature_names:
            feats.times_plus_equal(1, FeatureVector({'t='+cur_tag: 1}))
        if 'prev_tag' in self.feature_names:
            feats.times_plus_equal(1, FeatureVector({'ti='+cur_tag+"+ti-1="+pre_tag: 1}))
        if 'current_word' in self.feature_names:
            feats.times_plus_equal(1, FeatureVector({'t='+cur_tag+'+w='+self.inputs['tokens'][i]: 1}))
        return feats

class FeatureVector(object):

    def __init__(self, fdict):
        self.fdict = fdict

    def times_plus_equal(self, scalar, v2):
        """
        self += scalar * v2
        :param scalar: Double
        :param v2: FeatureVector
        :return: None
        """
        for key, value in v2.fdict.items():
            self.fdict[key] = scalar * value + self.fdict.get(key, 0)


    def dot_product(self, v2):
        """
        Computes the dot product between self and v2.  It is more efficient for v2 to be the smaller vector (fewer
        non-zero entries).
        :param v2: FeatureVector
        :return: Int
        """
        retval = 0
        for key, value in v2.fdict.items():
            retval += value * self.fdict.get(key, 0)
        return retval

    def write_to_file(self, filename):
        """
        Writes the feature vector to a file.
        :param filename: String
        :return: None
        """
        print('Writing to ' + filename)
        with open(filename, 'w', encoding='utf-8') as f:
            for key, value in self.fdict.items():
                f.write('{} {}\n'.format(key, value))


    def read_from_file(self, filename):
        """
        Reads a feature vector from a file.
        :param filename: String
        :return: None
        """
        self.fdict = {}
        with open(filename, 'r') as f:
            for line in f.readlines():
                txt = line.split()
                self.fdict[txt[0]] = float(txt[1])

#test_decoder()  # Uncomment to test the decoder on a simple example (see https://classes.soe.ucsc.edu/cse143/Winter20/assignments/A3_Debug_Example.pdf)
main_predict('ner.test', 'model.simple')  # Uncomment to predict on 'dev.ner' using the model 'model.simple' (need to implement 'decode' function)
#main_train()    # Uncomment to train a model (need to implement 'sgd' function)

Streaming output truncated to the last 5000 lines.
['<START>', 'O', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['<START>', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['<START>', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['<START>', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['<START>', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['<START>', 'I-ORG', 'O', '

In [ ]:
!cat "/content/gdrive/My Drive/CSE-143-A3/model" | awk '{print $2, $1}' | sort -gr > "/content/gdrive/My Drive/CSE-143-A3/model.sorted.txt"

In [ ]:
# accuracy:  37.70%; (non-O)
# accuracy:  88.02%; precision:  53.26%; recall:  37.38%; FB1:  43.93
#               LOC: precision:  86.50%; recall:  55.76%; FB1:  67.81  1074
#              MISC: precision:  54.36%; recall:  50.64%; FB1:  52.44  653
#               ORG: precision:  37.26%; recall:  44.93%; FB1:  40.74  1986
#               PER: precision:  32.89%; recall:   4.68%; FB1:   8.20  228